In [3]:
import os
import subprocess
import pandas as pd
import shutil

### Runner :

Calls a script file in a specified directory.

In [128]:
class RunnerVariants:
    def __init__(self,operand_sizes, 
                 script_dir,
                 threads=4,
                 backend_manager=None,
                 backend_commands=None):
        """
        This class handles the code generation and execution of the variant codes.
        The generated event data can be obtained as a pandas dataframe.

        Requirements:

        It is assumed that there exists a script file that generates variant codes
        for a given oopoerand sizes. The operand sizes are input as command line args
            e.g. run of script file: python generate.py 10 10 10 10 12

        After running the script file, inside the folder "experiments", which is in
        the same directory as the script file, an "argument folder" is generated,
        which contains the case_table, event_meta_table (i.e, the event table without actual run times)
        ,and a runner script as shown in the expample below:
            e.g. experiment/10_10_10_10_12/
                    case_table.csv
                    event_meta_table.csv
                    runner.jl

        'runner.jl' is the script that runs the experiments and generates a log file 'run_times.txt' (which is the
        event table with actual run times) in the "arguments folder"


        INPUT:

        name: Experiment name
        script_path: Path to the script file that generates variants
        args: operand sizes (or arguments to the script file)

        USECASE:
        If the behavior of the script is as said in the requirements, this class can
        call the scipt file and collects the eventlogs as a pandas dataframe, and
        if needed, can also clean the generated folders.

        """
        self.script_dir = script_dir
        self.operand_sizes = operand_sizes
        self.operands_dir = os.path.join(self.script_dir,
                                     "experiments",
                                     "_".join(self.operand_sizes))

        self.backend_manager = backend_manager
        self.backend_commands = backend_commands
        self.threads = threads

    def generate_variants_for_measurements(self, generation_script):
        """
        generates experiments for a given set of valid arguments
        that can be given as input to the script file.
            e.g. in,  python generate.py 10 10 10 10 12
            ['10','10','10','10','12'] would be the argument list.

        Output: Return code == 0 implies successful completion
        """
        script_path = os.path.join(self.script_dir, generation_script)
        args = self.operand_sizes + ["--threads={}".format(self.threads)]
        if not self.backend_manager:
            call = ["python", script_path] + args
            print(call)
            completed_proccess = subprocess.run(call)
            ret = completed_proccess.returncode
        else:
            cmd = self.backend_commands.build_cmd("python", script_path, " ".join(args))
            #print(cmd)
            stdout, ret = self.backend_manager.run_cmd(cmd)
            print(stdout.readlines())

        return ret

    def measure_variants(self, app, runner_script):
        """
        executes the runner file, which generates run_times.txt
        """
        runner_path = os.path.join(self.operands_dir, runner_script)
        if not self.backend_manager:
            if os.path.exists(self.operands_dir):
                print("Running Experiments locally")
                completed_proccess = subprocess.run([app, runner_path])
                if completed_proccess.returncode == 0:
                    print("Experiments completed locally")
                    return 0  # Ran experiment
        else:
            cmd = self.backend_commands.build_cmd(app, runner_path)
            stdout, ret = self.backend_manager.run_cmd(cmd)
            print(stdout.readlines())
            if ret == 0:
                print("Running experiments in the backend.")
                return 0

        return -1
    
    def generate_measurements_script(self, measurement_script, competing_variants, run_id, reps):
        pass

    def clean(self):
        """remove arguments folder"""
        if os.path.exists(self.operands_dir):
            shutil.rmtree(self.operands_dir)
        else:
            return -1



### Local measurements

In [129]:
operand_sizes = ["75","75","6","75","75"]
script_dir = "sample_generation/"
runner_local = RunnerVariants(operand_sizes, script_dir)

In [130]:
runner_local.generate_variants_for_measurements(generation_script="generate-variants-linnea.py")

['python', 'sample_generation/generate-variants-linnea.py', '75', '75', '6', '75', '75', '--threads=4']
New solution:.............2.02e+05
No further generation steps possible.
----------------------------------
Number of nodes:                 8
Solution nodes:                  1
Data:                     1.78e+04
Best solution:            2.02e+05
Intensity:                    11.4
Number of algorithms:            6
Generated Variants.


0

In [131]:
runner_local.measure_variants(app="julia", runner_script="runner.jl")

Running Experiments locally
Experiments completed locally


0

### Backend measurements

In [132]:
from backend_manager import BackendManager,Commands

In [133]:
bm = BackendManager(server="login18-1.hpc.itc.rwth-aachen.de", uname="as641651")
bm.connect()
cmds = Commands(source="~/.analyzer")

In [134]:
operand_sizes = ["75","75","6","75","75"]
script_dir = "sample_generation/"
generation_script = "generate-variants-linnea.py"
runner = RunnerVariants(operand_sizes, script_dir,backend_manager=bm, backend_commands=cmds)

In [135]:
ret = runner.generate_variants_for_measurements(generation_script=generation_script)

source ~/.analyzer; cd sample_generation; python generate-variants-linnea.py 75 75 6 75 75 --threads=4
['New solution:.............2.02e+05\n', 'No further generation steps possible.\n', '----------------------------------\n', 'Number of nodes:                 8\n', 'Solution nodes:                  1\n', 'Data:                     1.78e+04\n', 'Best solution:            2.02e+05\n', 'Intensity:                    11.4\n', 'Number of algorithms:            6\n', 'Generated Variants.\n']


In [136]:
ret

0

In [137]:
runner.measure_variants(app="julia", runner_script="runner.jl")

source ~/.analyzer; cd sample_generation/experiments/75_75_6_75_75; julia runner.jl 
[]
Running experiments in the backend.


0